### Import Libraries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

### Load Data

In [ ]:
# Load data
df = pd.read_csv('../dataset/data.csv', sep=';')
df.head()

In [ ]:
# Variables table
df_variables = pd.read_csv('../dataset/variables.csv')
df_variables.head(len(df_variables))

In [ ]:
# Data info
df.info()

### Target Variable

In [ ]:
# Target variable
df_target = pd.read_csv('../dataset/labels.csv')
df_target.head()

In [ ]:
# Value counts
df_target['y'].value_counts()

In [ ]:
# Normalised value counts
df_target['y'].value_counts(normalize=True)

In [ ]:
# Visualise the target variable
sns.countplot(x='y', data=df_target, hue='y', palette='Set1')
plt.show()

### Missing Values

In [ ]:
# Count missing values per column
df.isnull().sum()

In [ ]:
# Replace missing values

# Numerical columns has no missing values

# Categorical columns
df = df.fillna(value={'job' : 'missing'})
df = df.fillna(value={'education' : 'missing'})
df = df.fillna(value={'contact' : 'missing'})
df = df.fillna(value={'poutcome' : 'missing'})


In [ ]:
# Recount missing values per column
df.isnull().sum()

### Unique Values

In [ ]:
# Investigate unique values
df.nunique()

### EDA

In [ ]:
# Add the 'y' column from df_target as a new column in df
df['y'] = df_target['y']
df.head()

In [ ]:
# Categorical Variables
def explore_categorical(col_name):   
    print("{0} Summary".format(col_name))
    print("\n")
    
    # Print the count of each category
    print("{0} Counts".format(col_name))
    print(df[col_name].value_counts())
    print("\n")

    # Print the ratio of each category
    print("{0} Ratio".format(col_name))
    print(df[col_name].value_counts(normalize=True))
    print("\n")
    
    # Print counts of loan defaults grouped by the categorical column
    print("{0} Default Counts".format(col_name))
    print(df.groupby(col_name)['y'].value_counts().unstack(level=-1))
    print("\n")

    # Print ratio of loan defaults grouped by the categorical column
    print("{0} Default Ratio".format(col_name))
    print(df.groupby(col_name)['y'].value_counts(normalize=True).unstack(level=-1))
    print("\n")

    # Plot ratio of loan defaults by the categorical column
    sns.catplot(data=df,kind='count',x=col_name,hue='y')
    plt.show()

In [ ]:
# Explore job column
explore_categorical('education')

In [ ]:
# Continous Variables 
def explore_continuous(col_name):
    # Print statistical summary
    print("{0} Summary".format(col_name))
    print("\n")
    print(df[col_name].describe())
    print("\n")

    # Look at boxplot
    sns.boxplot(x=col_name, data=df)
    plt.show()

    # Look at the distribution
    sns.distplot(df[col_name], hist=False)
    plt.show()

    # Grouping with the target variable 
    print("{0} Grouped Summary".format(col_name))
    print("\n")
    print(df.groupby('y')[col_name].describe())

    # Look at grouped boxplot 
    sns.boxplot(x=col_name, y='y', data=df, orient="h")
    plt.show()

In [ ]:
# Explore age column
explore_continuous('pdays') 

In [ ]:
# Binary Variables
explore_categorical('housing')

### Normalisation

In [ ]:
## Numerical values
numerical_columns = df_variables[df_variables['type'] == 'Integer']['name'].tolist()
print(numerical_columns)
df[numerical_columns].boxplot()
plt.title('Prior to Scaling')
plt.show()

mm_scalar = MinMaxScaler()
df[numerical_columns] = mm_scalar.fit_transform(df[numerical_columns])

df[numerical_columns].boxplot()
plt.title('After Scaling')
plt.show()



In [ ]:
## Binary 
binary_columns = df_variables[df_variables['type'] == 'Binary']['name'].tolist()
print(binary_columns)

df[binary_columns] = df[binary_columns].apply(lambda col: col.map({'yes': 1, 'no': 0}))
df.head()


In [ ]:
## Categorical (Ordinal data)
ordinal_column = 'education'
categories = sorted(df[ordinal_column].unique())

# Ordinal-encoding
df[ordinal_column] = df[ordinal_column].apply(lambda x: (categories.index(x)) / (len(categories) - 1))
 
df.head()

print(df[ordinal_column].value_counts())



In [ ]:
## Categorical (Nominal) - One hot encoding  
nominal_column = df_variables[
    (df_variables['type'] == 'Categorical') & 
    (df_variables['name'] != 'education')
]['name'].tolist()
print(nominal_column)

### Train/Validation/Test Split

### ANN

### Model Evaluation

In [ ]:
# Confusion Matrix
# Precision
# Recall
# F1 Score

In [ ]:
# Decaying learning rate and non decaying learning rate